<a href="https://colab.research.google.com/github/daredevil007-aks/Forex_prediction/blob/main/model_trainer_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!free -h

               total        used        free      shared  buff/cache   available
Mem:            12Gi       3.4Gi       7.2Gi       1.0Mi       2.1Gi       9.0Gi
Swap:             0B          0B          0B


In [ ]:
import psutil
import os

def print_memory():
    pid = os.getpid()
    py = psutil.Process(pid)
    memoryUse = py.memory_info()[0] / 2.**30  # memory use in GB
    print(f'Memory usage: {memoryUse:.2f} GB')

print_memory()


In [15]:
import pandas as pd
import numpy as np
import time
from sklearn.preprocessing import MinMaxScaler

# Load data
data = pd.read_csv('DAT_MS_EURUSD_M1_202407.csv')

In [16]:

import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Convert `Datetime` to datetime format
data['Datetime'] = pd.to_datetime(data['Datetime'], unit='ms')

# Sort data by datetime just in case
data = data.sort_values(by='Datetime')

# Use only the 'Close' prices for this example
prices = data['Close'].values.reshape(-1, 1)

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_prices = scaler.fit_transform(prices)

In [17]:
look_back = 24 * 60  # 5 months of minute data (approx. 30 days per month) -> changing to 24 hours for testing hptuning
predict_forward = 5  # Predict next 5 minutes

# Ensure there are enough data points
if len(scaled_prices) < look_back + predict_forward:
    raise ValueError("Not enough data points for the given look_back period.")

# Create dataset
def create_minute_dataset(dataset, look_back, predict_forward=5):
    X, Y = [], []
    for i in range(len(dataset) - look_back - predict_forward):
        # time.sleep(1)
        a = dataset[i:(i + look_back), 0]
        X.append(a)
        Y.append(dataset[(i + look_back):(i + look_back + predict_forward), 0])
    return np.array(X), np.array(Y)

X, Y = create_minute_dataset(scaled_prices, look_back, predict_forward)

In [18]:
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

In [19]:
!pip install scikeras[tensorflow] --quiet


In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import GridSearchCV

# Model creation function with **kwargs to accept dynamic parameters
def create_model(units=50, optimizer='adam', **kwargs):
    model = Sequential()
    model.add(LSTM(units, return_sequences=True, input_shape=(look_back, 1)))
    model.add(LSTM(units, return_sequences=False))
    model.add(Dense(25))
    model.add(Dense(predict_forward))

    model.compile(optimizer=optimizer, loss='mean_squared_error')
    return model

# Create the KerasRegressor with model creation function
model = KerasRegressor(model=create_model, verbose=0)

# Define the grid search parameters
param_grid = {
    'model__units': [50, 100],         # Use 'model__' prefix for parameters of the model function
    'model__optimizer': ['adam', 'rmsprop'],
    'epochs': [10, 20],
    'batch_size': [16, 32]
}

# Create GridSearchCV
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)

# Fit the model
X_small = X[:1000]
Y_small = Y[:1000]
grid_result = grid.fit(X_small, Y_small)


# Output the best parameters
print(f"Best parameters: {grid_result.best_params_}")


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Best parameters: {'batch_size': 16, 'epochs': 20, 'model__optimizer': 'adam', 'model__units': 100}


In [ ]:
from sklearn.model_selection import RandomizedSearchCV

param_grid = {
    'units': [50, 100, 150],
    'batch_size': [1, 5, 10],
    'epochs': [10, 20, 30],
    'optimizer': ['adam', 'rmsprop']
}

# Create RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=model,
                                   param_distributions=param_grid,
                                   n_iter=10,
                                   cv=3,
                                   verbose=2,
                                   random_state=42,
                                   n_jobs=-1)

# Fit RandomizedSearchCV
random_search.fit(X_small, Y_small)

In [ ]:
print("Best Parameters: ", random_search.best_params_)
print("Best Score: ", random_search.best_score_)
